# This is the project in Optimization 1

In [ ]:
from functions import *
import numpy as np
import matplotlib.pyplot as plt
# from numpy.random import default_rng
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.datasets import load_iris
from scipy.optimize import approx_fprime

In [ ]:
np.random.seed(42)
w = np.array([1.,1.])
b = 1.

n = 10
n_A = np.random.randint(0,n)
n_B = n-n_A
margin = 5.e-1
listA, listB = TestLinear(w,b,n_A,n_B,margin)

x = np.concatenate((np.array(listA),np.array(listB)))

y = np.concatenate((np.ones(n_A), -np.ones(n_B)))

plt.scatter(x[:, 0], x[:, 1], c=y, cmap='coolwarm', edgecolors='k')
plt.title("Data points")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")

plt.show()


In [ ]:

alpha0 = np.zeros(n_A+n_B)
tau = 0.1
niter = 1000
C = 3
G = pairwise_kernels(x, metric = kernal_linear)  

alpha = gradient_descent(alpha0, G, y, tau0=tau, niter=niter, C=C, tol=1e-7, projection=projection)

w, b = w_b(alpha, y,x ,  C=C)
plot_solution(x, y, w, b)


C = 45
G = pairwise_kernels(x, metric = kernal_linear)  

alpha = gradient_descent(alpha0, G, y, tau0=tau, niter=niter, C=C, tol=1e-7, projection=projection)

w, b = w_b(alpha, y,x ,  C=C)
plot_solution(x, y, w, b)

In [ ]:

alpha0 = np.zeros(n_A+n_B)
tau = 1
niter = 1000
C = 50
G = pairwise_kernels(x, metric = kernal_linear)


alpha = gradient_descent(alpha0, G, y, tau0=tau, niter=niter, C=C, tol=1e-7, projection=projection)
alphagrad, fks = gradient_descent_linesearch(alpha0, G, y, tau0=tau, niter=niter, C=C, tol=1e-7, project=projection)

w, b = w_b(alpha, y,x ,  C=C)
plt.title("Gradient descent without linesearch")
plot_solution(x, y, w, b)

w, b = w_b(alphagrad, y,x ,  C=C)
plt.title("Gradient descent with linesearch")
plot_solution(x, y, w, b)



In [ ]:

plot_db(x, y, alpha, ker = kernal_linear, C=C)
plot_db(x, y, alphagrad, ker = kernal_linear, C=C)


### Testing the different kernels

In [ ]:
niter = 1000
C = 5

G = pairwise_kernels(x, metric = kernal_gaussian, sigma=1)
alpha = gradient_descent(alpha0, G, y, tau, niter, C)
# w, b = w_b(alpha, y, x, C=C)
# plot_solution(x, y, w, b)
plot_db(x, y, alpha, ker = kernal_gaussian, C=C)    

G = pairwise_kernels(x, metric = kernal_inv_multiquadratic, sigma=1)
alpha = gradient_descent(alpha0, G, y, tau, niter, C)
# w, b = w_b(alpha, y, x, C=C)
# plot_solution(x, y,  w, b)
plot_db(x, y, alpha, ker = kernal_inv_multiquadratic, C=C)

G = pairwise_kernels(x, metric = kernal_laplacian, sigma=1)
alpha = gradient_descent(alpha0, G, y, tau, niter, C)
# w, b = w_b(alpha, y, x, C=C)
# plot_solution(x, y,  w, b)
plot_db(x, y, alpha, ker = kernal_laplacian, C=C)



### Testing with random initial contitions

In [ ]:
alpha0 = np.random.rand((n_A+n_B))
ker = kernal_gaussian
G = pairwise_kernels(x, metric = ker)  

niter = 1000
C = 5
alpha_line, fk = gradient_descent_linesearch(alpha0, G, y, tau, niter, C = C, L=10, tol=1e-7)
alpha_grad = gradient_descent(alpha0, G, y, tau, niter, C=C, tol=1e-7)

plot_db(x, y, alpha_line, ker, C=C)
plot_db(x, y, alpha_grad, ker, C=C)


### Using gradient descent on the iris dataset

In [ ]:
iris = load_iris()
X = iris.data[:, :2][0:100]  # We'll use only the first two features for visualization
Y = iris.target[0:100]  
Y[Y == 0] = -1  # Convert to -1 and 1 for SVM
Y[Y == 1] = 1


plt.figure(figsize=(10, 5))
plt.scatter(X[:, 0], X[:, 1], c=Y, cmap=plt.cm.Set1, s=50, edgecolors='k')
plt.title('Iris Dataset (First Two Features)')
plt.xlabel('Sepal Length (cm)')
plt.ylabel('Sepal Width (cm)')
plt.show()

alpha0 = np.zeros(len(Y))
ker = kernal_gaussian
G = pairwise_kernels(X, metric=ker)
C= 100

alpha, f = gradient_descent_linesearch(alpha0, G, Y, tau0=tau, niter=niter, C=C, L=10, tol=1e-7, project=projection)

plot_db(X, Y, alpha, ker=ker, C=C)

### Bound constrained lagrangian method

In [ ]:
d = len(x[0])
M = len(x)
epsilon = 1e-7

C = 10

startpunkt = np.ones(d+1+2*M)
lambd_0 = np.ones(M)
mu_0 = 10
tol_1 = 1e-7
tol_2 = 1e-7
maxiter = 1000

AL_par = [lambd_0, mu_0, d, M, x, y, C]
gradAL_par = [lambd_0, mu_0, d, M, x, y, C]
constr_par = [x, y]

lower_bound = np.append([- np.inf]*(d+1), [0]*2*M)
upper_bound = [np.inf]*(d+1+2*M)
project_par = [lower_bound, upper_bound]
                                         
linesearch_par = [lambd_0, mu_0, d, M, x, y, C]

BCLM(startpunkt, lambd_0, mu_0, tol_1, tol_2, maxiter, AL, AL_par, constraints, constr_par, grad_AL, gradAL_par, projection_AL, project_par, linesearch_AL, linesearch_par)